In [20]:
import re
import pandas as pd
from bertopic import BERTopic

In [24]:
df_candidate_tweets = pd.read_csv('../data/twitter_data/zemmour.csv', encoding='utf-8')
# df_candidate_tweets.dropna(inplace=True)

In [44]:
# preparing tweet texts
import preprocessing as pr

df_candidate_tweets['preprocessed_tweet'] = df_candidate_tweets.tweet_text.apply(pr.preprocess_tweet)
df_candidate_tweets['basic_cleaning'] = df_candidate_tweets.tweet_text.apply(pr.basic_clean)

df_candidate_tweets.basic_cleaning = df_candidate_tweets.basic_cleaning.astype('string')
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: re.sub(r"http\S+", "", row.basic_cleaning).lower(), 1)
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.basic_cleaning.split())), 1)
df_candidate_tweets.basic_cleaning = df_candidate_tweets.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.basic_cleaning).split()), 1)

# len(df_candidate_tweets)

In [45]:
# filtering date
df_candidate_tweets.time = pd.to_datetime(df_candidate_tweets['time'], dayfirst=True)
df_candidate_tweets = df_candidate_tweets[df_candidate_tweets['time'] > '2019-07-01']

In [46]:
# saving text and time in 2 different lists
timestamps = df_candidate_tweets.time.to_list()
tweets = df_candidate_tweets.basic_cleaning.to_list()

In [47]:
# getting topics
topic_model = BERTopic(language="multilingual")
topics, probs = topic_model.fit_transform(tweets)

In [51]:
# finding topic's distribbution over time
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamps, nr_bins=20)
# visualization
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'hovertext': [<b>Topic 1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , , <b>Topic
                            1</b><br>Words: strike, , , , ],
              'marker': {'color': '#E69F00'},
              'mode': 'lines',
              'name': '1_strike___',
              'type': 'scatter',
              'x': array([datetime.datetime(2021, 8, 1, 7, 22, 23, 649999),
                          datetime.datetime(2021, 8, 5, 17, 4, 32, 500000),
                          datetime.datetime(2021, 8, 10, 0, 42, 20),
                          datetime.datetime(2021, 8, 14, 8, 20, 7, 500000),
                          datetime.datetime(2021, 8, 18, 15, 57, 55),
                          datetime.datetime(2021, 8, 22, 23, 35, 42, 500000),
                          datetime.datetime(2021, 9, 22, 5, 0, 15),
                          datetime.datetime(2021, 9, 26, 12, 38, 2, 500000),
                          datetime.datetime(2021, 9, 30, 20, 15, 50),
                          datetime.datetime(2021, 10, 5, 3, 53, 37, 500000),
                          datetime.datetime(2021, 10, 9, 11, 31, 25),
                          datetime.datetime(2021, 10, 13, 19, 9, 12, 500000),
                          datetime.datetime(2021, 10, 18, 2, 47),
                          datetime.datetime(2021, 10, 22, 10, 24, 47, 500000)], dtype=object),
              'y': array([1, 2, 4, 3, 3, 1, 2, 2, 2, 2, 2, 5, 4, 2])},
             {'hoverinfo': 'text',
              'hovertext': [<b>Topic -1</b><br>Words: écrira, chapitre, toulon,
                            de, ajaccio, <b>Topic -1</b><br>Words: informations,
                            écrira, chapitre, de, prochain, <b>Topic
                            -1</b><br>Words: écrira, chapitre, ajaccio, corse, de,
                            <b>Topic -1</b><br>Words: écrira, chapitre, toutes,
                            nîmes, de, <b>Topic -1</b><br>Words: écrira, chapitre,
                            ici, rouen, de, <b>Topic -1</b><br>Words: informations,
                            écrira, chapitre, prochain, biarritz],
              'marker': {'color': '#56B4E9'},
              'mode': 'lines',
              'name': '-1_voir_chapitre_ici_de',
              'type': 'scatter',
              'x': array([datetime.datetime(2021, 9, 4, 22, 29, 5),
                          datetime.datetime(2021, 9, 26, 12, 38, 2, 500000),
                          datetime.datetime(2021, 10, 5, 3, 53, 37, 500000),
                          datetime.datetime(2021, 10, 9, 11, 31, 25),
                          datetime.datetime(2021, 10, 18, 2, 47),
                          datetime.datetime(2021, 10, 22, 10, 24, 47, 500000)], dtype=object),
              'y': array([1, 1, 1, 1, 1, 1])}],
    'layout': {'height': 450,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': '<b>Global Topic Representation'}},
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Topics over Time',
                         'x': 0.4,
        